# Análisis integral de conjunto de datos para la industria y creación de un panel de visualización y despliegue de modelos

El objetivo de este notebook es desarrollar la aplicación interactiva de analítica industrial que tenga 
visualización, modelado y despliegue de inteligencia artificial. El dataset seleccionado para ello es Tennessee Eastman Process Simulation Dataset, una simulación de un proceso químico industrial, diseñado como benchmark para estudiar control, detección de fallos y monitoreo de procesos. Por lo que incluye datos de operación normal y con múltiples fallos simulados, permitiendo evaluar algoritmos de diagnóstico y predicción en entornos realistas.

## Preparación de Datos

Empezamos con la carga, exploración, limpieza y preprocesamiento de los datos, dejando el dataset listo para las siguientes implementaciones. 

In [1]:
# Importación de librerías
!pip install pyreadr
!pip install scikit-learn

import pyreadr
import pandas as pd
from sklearn.preprocessing import StandardScaler

### 1. Función de carga de datos

Los archivos originales están en formato `.RData`. Asi que para leerlos utilizamos la librería **pyreadr**, que devuelve un diccionario con los objetos contenidos en el archivo. 
Definimos una función que carga el archivo y devuelve el DataFrame correspondiente.

In [2]:
def load_rdata(file_path):
    result = pyreadr.read_r(file_path)
    df_name = list(result.keys())[0]
    return result[df_name]

### 2. Carga de datasets

El benchmark TEP incluye cuatro conjuntos principales:

- **Fault Free Training**: simulaciones sin fallos para entrenamiento.  
- **Fault Free Testing**: simulaciones sin fallos para validación.  
- **Faulty Training**: simulaciones con distintos fallos para entrenamiento.  
- **Faulty Testing**: simulaciones con fallos para validación.

In [3]:
datasets = {
    "Fault Free Training": load_rdata("TEP_FaultFree_Training.RData"),
    "Fault Free Testing": load_rdata("TEP_FaultFree_Testing.RData"),
    "Faulty Training": load_rdata("TEP_Faulty_Training.RData"),
    "Faulty Testing": load_rdata("TEP_Faulty_Testing.RData")
}

### 3. Definición de variables

El proceso TEP incluyen dos grupos de variables, que vamos a definir para facilitar su acceso cuando sea necesario. 

- 41 variables de proceso, mediciones de sensores dentro de la planta química simulada: (`xmeas_1` a `xmeas_41`)  
- 11 variables de actuadores, variables manipuladas por los controladores de la planta : (`xmv_1` a `xmv_11`)

In [4]:
process_vars = [f"xmeas_{i}" for i in range(1, 42)]
actuator_vars = [f"xmv_{i}" for i in range(1, 12)]

### 4. Información general de cada dataset

Enseñamos en formato DataFrame:
- Tamaño del dataset (df.shape)
- Número de simulaciones (simulationRun.nunique())
- Rango temporal de muestras (sample.min() – sample.max())
- Fallos presentes (si existe la columna faultNumber)
- Conteo de fallos (solo en los datasets Faulty)

In [5]:
import pandas as pd

# Crear lista de resúmenes
resumen = []
for name, df in datasets.items():
    info = {
        "Dataset": name,
        "Tamaño": df.shape,
        "Nº simulaciones": df['simulationRun'].nunique(),
        "Rango muestras": f"{df['sample'].min()} - {df['sample'].max()}"
    }
    if 'faultNumber' in df.columns:
        info["Fallos presentes"] = sorted(df['faultNumber'].unique())
        if name.startswith("Faulty"):
            info["Conteo fallos"] = df['faultNumber'].value_counts().sort_index().to_dict()
    resumen.append(info)

# Convertir a DataFrame
tabla_resumen = pd.DataFrame(resumen)

# Mostrar tabla resumen
print("\n=== Resumen de datasets ===")
display(tabla_resumen)

# Mostrar head de cada dataset
for name, df in datasets.items():
    print(f"\n--- Head de {name} ---")
    display(df.head())


=== Resumen de datasets ===


,Dataset,Tamaño,Nº simulaciones,Rango muestras,Fallos presentes,Conteo fallos
0,Fault Free Training,"(250000, 55)",500,1 - 500,[0.0],NaN
1,Fault Free Testing,"(480000, 55)",500,1 - 960,[0],NaN
2,Faulty Training,"(5000000, 55)",500,1 - 500,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1: 250000, 2: 250000, 3: 250000, 4: 250000, 5..."
3,Faulty Testing,"(9600000, 55)",500,1 - 960,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1: 480000, 2: 480000, 3: 480000, 4: 480000, 5..."



--- Head de Fault Free Training ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461



--- Head de Fault Free Testing ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0,1.0,1,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,0,1.0,2,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,0,1.0,3,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,0,1.0,4,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,0,1.0,5,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681



--- Head de Faulty Training ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,1,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,1,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,1,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,1,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461



--- Head de Faulty Testing ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1,1.0,1,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,1,1.0,2,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,1,1.0,3,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,1,1.0,4,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,1,1.0,5,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681


### 5. Estadísticas descriptivas generales

Calculamos estadísticas básicas (media, desviación estándar, mínimo, percentiles y máximo) para todas las variables de proceso y actuadores en cada dataset, mostrando el resultado en una tabla para cada dataset.


In [6]:
import pandas as pd

# Crear lista de resúmenes estadísticos
estadisticas = []
for name, df in datasets.items():
    stats = df[process_vars + actuator_vars].describe().T[['mean','std','min','25%','50%','75%','max']]
    stats.insert(0, "Variable", stats.index)  
    stats.insert(0, "Dataset", name)          
    estadisticas.append(stats)

# Concatenar todas las tablas en una sola
tabla_estadisticas = pd.concat(estadisticas, ignore_index=True)

# Mostrar tabla resumen
print("\n=== Estadísticas generales de todos los datasets ===")
display(tabla_estadisticas)

# Mostrar head de cada dataset
for name, df in datasets.items():
    print(f"\n--- Head de {name} ---")
    display(df[process_vars + actuator_vars].head())


=== Estadísticas generales de todos los datasets ===


,Dataset,Variable,mean,std,min,25%,50%,75%,max
0,Fault Free Training,xmeas_1,0.250482,0.030873,0.122450,0.22977,0.25051,0.2712,0.39174
1,Fault Free Training,xmeas_2,3663.785706,34.006357,3511.800000,3640.80000,3663.70000,3686.8000,3808.40000
2,Fault Free Training,xmeas_3,4508.819790,39.215420,4336.900000,4482.40000,4508.80000,4535.3000,4683.70000
3,Fault Free Training,xmeas_4,9.347101,0.085721,8.972700,9.28930,9.34710,9.4051,9.72980
4,Fault Free Training,xmeas_5,26.902196,0.211485,25.951000,26.75800,26.90200,27.0460,27.81800
...,...,...,...,...,...,...,...,...,...
203,Faulty Testing,xmv_7,38.046287,2.937202,22.288000,36.00000,38.11200,39.9610,54.23000
204,Faulty Testing,xmv_8,46.422670,2.341916,33.369000,44.85600,46.31900,48.0080,60.21600
205,Faulty Testing,xmv_9,49.641559,17.517283,-0.683420,45.38500,48.04900,51.4460,100.58000
206,Faulty Testing,xmv_10,42.035994,11.875982,-0.515650,40.61600,41.19100,41.8530,100.61000



--- Head de Fault Free Training ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,0.33818,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,75.000,120.41,0.33620,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,74.771,120.42,0.33563,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,75.224,120.39,0.33553,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,75.388,120.39,0.32632,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461



--- Head de Fault Free Testing ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,74.750,120.41,0.33642,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,75.126,120.38,0.33801,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,74.540,120.38,0.33702,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,74.861,120.38,0.33648,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,74.380,120.40,0.32114,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681



--- Head de Faulty Training ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,0.33818,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,75.000,120.41,0.33620,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,74.771,120.42,0.33563,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,75.224,120.39,0.33553,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,75.388,120.39,0.32632,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461



--- Head de Faulty Testing ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,74.750,120.41,0.33642,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,75.126,120.38,0.33801,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,74.540,120.38,0.33702,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,74.861,120.38,0.33648,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,74.380,120.40,0.32114,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681


### 6. Tabla resumen de simulaciones y fallos

Sintetizamos la información clave de cada dataset en una tabla.

In [7]:
summary = []
for name, df in datasets.items():
    entry = {
        "Dataset": name,
        "Simulaciones": df['simulationRun'].nunique(),
        "Muestras": df['sample'].nunique()
    }
    if 'faultNumber' in df.columns:
        entry["Fallos únicos"] = df['faultNumber'].nunique()
    else:
        entry["Fallos únicos"] = 0
    summary.append(entry)

summary_df = pd.DataFrame(summary)
summary_df

,Dataset,Simulaciones,Muestras,Fallos únicos
0,Fault Free Training,500,500,1
1,Fault Free Testing,500,960,1
2,Faulty Training,500,500,20
3,Faulty Testing,500,960,20


### 7. Preprocesamiento: Normalización y variables derivadas

Normalizamos los datos y creamos variables derivadas. Un ejemplo de esto puede ser lo siguiente:
- **Normalización** con `StandardScaler`. Con esto aseguramos que todas las variables estén en la misma escala, algo crítico para algoritmos basados en distancia o redes neuronales. 
- **Variables derivadas**: medias móviles y diferencias temporales. Con lo que demostramos lo significativo que puede ser para dataset con información de tendencias y dinámicas, sin alterar la estructura original.

In [9]:
preprocessed_datasets = {}
scaler = StandardScaler()

for name, df in datasets.items():
    df_copy = df.copy()
    
    # Normalización
    df_copy[process_vars + actuator_vars] = scaler.fit_transform(df_copy[process_vars + actuator_vars])
    
    # Variables derivadas (ejemplo con primeras 5 variables)
    for var in process_vars[:5]:
        df_copy[f"{var}_ma5"] = df_copy[var].rolling(window=5).mean()
        df_copy[f"{var}_diff"] = df_copy[var].diff()
    
    preprocessed_datasets[name] = df_copy

preprocessed_datasets["Fault Free Training"].head(10)

MemoryError: Unable to allocate 3.79 GiB for an array with shape (53, 9600000) and data type float64

## Análisis exploratorio

En esta sección vamos a realizar un analisis más profundo para poder encontrar relaciones entre variables. Haremos uso de estadística descriptiva y gráficos como herramientas.

In [ ]:
# Instalación de librería para leer archivos RData
!pip install pyreadr pandas numpy

import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Configuración de visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### 1.Preparación de Datos y Verificación
Se verifica la estructura, tipos de datos y la ausencia de valores nulos (confirmando el análisis inicial).

In [ ]:
# 1. Distribución de faultNumber (Multi-clase)
plt.figure(figsize=(10, 5))
sns.countplot(x='faultNumber', data=df_train_raw)
plt.title('Distribución de Frecuencia por Tipo de Fallo en Entrenamiento (0 = Normal)')
plt.show()

# 2. Distribución Binaria (Fallo vs. Normal)
fault_counts = df_train_raw['fault_present'].value_counts()
plt.figure(figsize=(4, 4))
fault_counts.plot.pie(
    autopct='%1.1f%%', 
    labels=['Fallo (1)', 'Normal (0)'] if fault_counts.iloc[0] == 1 else ['Normal (0)', 'Fallo (1)'], 
    colors=['#FF5722', '#4CAF50']
)
plt.ylabel('')
plt.title('Proporción de Muestras en Entrenamiento: Normal vs. Fallo')
plt.show()

La distribución de faultNumber muestra que todas las clases de fallo están representadas, que sugieren una simulación balanceada.

A nivel binario, el conjunto de entrenamiento presenta una fuerte desbalance, con un 95.2% de muestras normales y solo 4.8% con fallo, lo que indica que será necesario aplicar técnicas para manejar el desbalance al entrenar modelos de clasificación.

### 2. Análisis Exploratorio de Datos (EDA)
El EDA se realizará principalmente sobre el conjunto de entrenamiento para entender la dinámica del proceso.

In [ ]:
# Seleccionar solo variables de proceso/manipuladas
process_vars = [col for col in df_train_raw.columns if 'xmeas' in col or 'xmv' in col]
cols_to_corr = process_vars + ['fault_present']

corr_matrix = df_train_raw[cols_to_corr].corr()

# Mostrar Heatmap de correlación (limitado a las 10 variables más correlacionadas con el fallo)
high_corr_features = corr_matrix['fault_present'].abs().sort_values(ascending=False).index[:11]
plt.figure(figsize=(10, 8))
sns.heatmap(
    df_train_raw[high_corr_features].corr(), 
    annot=True, 
    fmt=".2f", 
    cmap='RdBu_r'
)
plt.title('Mapa de Calor de Correlación (Top 10 Features vs. Fallo) - Entrenamiento')
plt.show()

# Conclusión: Se identifican los predictores más fuertes para la fase de modelado.

Las correlaciones con la variable fault_present son muy bajas, indicando que ningún sensor o variable manipulada presenta una relación lineal fuerte con la presencia de fallos. Por otro lado, si se muestra correlación entre otras variables, lo cual puede indicar redundancia en las mediciones. Para el proyecto, esto quiere decir que podemos aplicar selección de variables o reducción de dimensiones durante el modelado.

### 2.2. Análisis de Series Temporales (Ejemplo)

In [ ]:
# Seleccionar una simulación con un fallo conocido (ejemplo: la primera simulación con fallo)
simulation_run = df_train_raw[df_train_raw['fault_present'] == 1]['simulationRun'].iloc[0]
df_sim_fault = df_train_raw[df_train_raw['simulationRun'] == simulation_run].copy()

# Variable (ejemplo de xmeas_1)
key_variable = 'xmeas_1'

plt.figure(figsize=(12, 4))
sns.lineplot(x='sample', y=key_variable, data=df_sim_fault, hue='fault_present', style='fault_present', markers=False)

# Marcar el inicio de la inyección del fallo
fault_start_sample = df_sim_fault[df_sim_fault['fault_present'] == 1]['sample'].min()
if pd.notna(fault_start_sample):
    plt.axvline(x=fault_start_sample, color='r', linestyle='--', label='Inicio del Fallo')

plt.title(f'Serie Temporal de {key_variable} en Simulación {simulation_run} (Entrenamiento)')
plt.legend()
plt.show()

Aqui podemos ver que la variable xmeans_1 tiene un alto nivel de ruido, pero el valor de la clase fallo tiene valores ligeramente superiores.

El rango de variación del fallo (la sombra naranja) es mucho más amplio que la clase normal, lo que indica un aumento en la desviación e inestabilidad del proceso después del comienzo del fallo.